In [1]:
import pandas

casas = pandas.read_csv("Casas.csv")

columnas_analisis = [
    "MSZoning",
    "LotArea",
    "Street",
    "Neighborhood",
    "YearBuilt",
    "OverallCond",
    "ExterQual",
    "GrLivArea",
    "FullBath",
    "GarageArea",
    "BsmtCond",
    "FireplaceQu",
    "Electrical",
    "LotFrontage",
    "KitchenQual",
    "PavedDrive",
    "SalePrice",
]

casas_analisis = casas[columnas_analisis]

reporte = casas_analisis.isnull().sum().reset_index()
reporte.columns=["Columna", "Nulos"]
reporte[reporte["Nulos"] > 0]

,Columna,Nulos
10,BsmtCond,37
11,FireplaceQu,690
12,Electrical,1
13,LotFrontage,259


In [127]:
casas_analisis["BsmtCond"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1460 entries, 0 to 1459
Series name: BsmtCond
Non-Null Count  Dtype 
--------------  ----- 
1423 non-null   object
dtypes: object(1)
memory usage: 11.5+ KB


In [128]:
casas_analisis["BsmtCond"].value_counts()

BsmtCond
TA    1311
Gd      65
Fa      45
Po       2
Name: count, dtype: int64

In [137]:
def missforest_multivariable(data, target_column):
    import pandas as pd
    import numpy as np
    from sklearn.experimental import enable_iterative_imputer
    from sklearn.impute import IterativeImputer
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.preprocessing import LabelEncoder

    df = data.copy()
    encoders = {}

    # Codifica todas las columnas categóricas como enteros (temporalmente)
    for col in df.columns:
        if df[col].dtype == "object" or df[col].dtype.name == "category":
            le = LabelEncoder()
            not_null = df[col].notnull()
            le.fit(df[col][not_null].astype(str))
            df.loc[not_null, col] = le.transform(df[col][not_null].astype(str))
            df.loc[~not_null, col] = np.nan  # mantiene NaNs
            encoders[col] = le

    # Asegura que todo sea numérico
    df = df.apply(pd.to_numeric, errors="coerce")

    # Imputación con todas las columnas
    imputer = IterativeImputer(
        estimator=RandomForestRegressor(n_estimators=100, random_state=0),
        max_iter=10,
        random_state=0
    )

    df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

    # Decodifica sólo la columna objetivo
    if target_column in encoders:
        col_imputed = df_imputed[target_column].round().astype(int)
        decoded = encoders[target_column].inverse_transform(col_imputed)
        return decoded
    else:
        return df_imputed[target_column]

In [138]:
casas_analisis.loc[:, "BsmtCond - Imputado"] = missforest_multivariable(casas_analisis, "BsmtCond")

casas_analisis[["BsmtCond", "BsmtCond - Imputado"]].sort_values("BsmtCond")

,BsmtCond,BsmtCond - Imputado
174,Fa,Fa
1014,Fa,Fa
789,Fa,Fa
383,Fa,Fa
124,Fa,Fa
...,...,...
1216,NaN,TA
1218,NaN,TA
1232,NaN,TA
1321,NaN,TA


In [139]:
casas_analisis["BsmtCond"].value_counts()

BsmtCond
TA    1311
Gd      65
Fa      45
Po       2
Name: count, dtype: int64

In [140]:
casas_analisis["BsmtCond - Imputado"].value_counts()

BsmtCond - Imputado
TA    1348
Gd      65
Fa      45
Po       2
Name: count, dtype: int64

In [141]:
casas_analisis.isnull().mean().sort_values(axis=0, ascending=False)

FireplaceQu               0.472603
LotFrontage               0.177397
BsmtCond                  0.025342
Electrical                0.000685
MSZoning                  0.000000
BsmtCond - Imputado       0.000000
SalePrice                 0.000000
PavedDrive                0.000000
KitchenQual               0.000000
GarageArea                0.000000
LotArea                   0.000000
FullBath                  0.000000
GrLivArea                 0.000000
ExterQual                 0.000000
OverallCond               0.000000
YearBuilt                 0.000000
Neighborhood              0.000000
Street                    0.000000
FireplaceQu - Imputado    0.000000
dtype: float64

In [142]:
casas_analisis["FireplaceQu"].value_counts()

FireplaceQu
Gd    380
TA    313
Fa     33
Ex     24
Po     20
Name: count, dtype: int64

In [143]:
casas_analisis.loc[:, ["FireplaceQu - Imputado"]] = missforest_multivariable(casas_analisis, "FireplaceQu")

In [144]:
casas_analisis["FireplaceQu - Imputado"].value_counts()

FireplaceQu - Imputado
Po    710
Gd    380
TA    313
Fa     33
Ex     24
Name: count, dtype: int64

In [3]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [4]:
%%R
install.packages("missForest")
library(missForest)

--- Please select a CRAN mirror for use in this session ---

The downloaded binary packages are in
	/var/folders/zr/py0pd6bs6gnfzg9ljbgr9v0c0000gn/T//Rtmpb81WQF/downloaded_packages


probando la URL 'https://cran.itam.mx/bin/macosx/big-sur-arm64/contrib/4.4/missForest_1.5.tgz'
Content type 'application/x-gzip' length 343906 bytes (335 KB)
downloaded 335 KB



In [5]:
import numpy

columnas_imputar = [
    "FireplaceQu",
    "LotFrontage",
    "BsmtCond",
    "Electrical",
]

for columna in columnas_imputar:
    print(casas_analisis[columna].dtype, casas_analisis[columna].dtype == object)
    if casas_analisis[columna].dtype == object:
        casas_analisis.loc[:, columna] = casas_analisis[columna].fillna("NA")
    elif casas_analisis[columna].dtype == numpy.float64:
        casas_analisis.loc[:, columna] = casas_analisis[columna].fillna(0.0)

casas_analisis[columnas_imputar]

object True
float64 False
object True
object True


,FireplaceQu,LotFrontage,BsmtCond,Electrical
0,NA,65.0,TA,SBrkr
1,TA,80.0,TA,SBrkr
2,TA,68.0,TA,SBrkr
3,Gd,60.0,Gd,SBrkr
4,TA,84.0,TA,SBrkr
...,...,...,...,...
1455,TA,62.0,TA,SBrkr
1456,TA,85.0,TA,SBrkr
1457,Gd,66.0,Gd,SBrkr
1458,NA,68.0,TA,FuseA


In [6]:
%R -i casas_analisis

In [9]:
%%R

head(casas_analisis)

  MSZoning LotArea Street Neighborhood YearBuilt OverallCond ExterQual
0       RL    8450   Pave      CollgCr      2003           5        Gd
1       RL    9600   Pave      Veenker      1976           8        TA
2       RL   11250   Pave      CollgCr      2001           5        Gd
3       RL    9550   Pave      Crawfor      1915           5        TA
4       RL   14260   Pave      NoRidge      2000           5        Gd
5       RL   14115   Pave      Mitchel      1993           5        TA
  GrLivArea FullBath GarageArea BsmtCond FireplaceQu Electrical LotFrontage
0      1710        2        548       TA          NA      SBrkr          65
1      1262        2        460       TA          TA      SBrkr          80
2      1786        2        608       TA          TA      SBrkr          68
3      1717        1        642       Gd          Gd      SBrkr          60
4      2198        2        836       TA          TA      SBrkr          84
5      1362        1        480       TA       

In [10]:
%%R
str(casas_analisis)

'data.frame':	1460 obs. of  17 variables:
 $ MSZoning    : chr  "RL" "RL" "RL" "RL" ...
 $ LotArea     : int  8450 9600 11250 9550 14260 14115 10084 10382 6120 7420 ...
 $ Street      : chr  "Pave" "Pave" "Pave" "Pave" ...
 $ Neighborhood: chr  "CollgCr" "Veenker" "CollgCr" "Crawfor" ...
 $ YearBuilt   : int  2003 1976 2001 1915 2000 1993 2004 1973 1931 1939 ...
 $ OverallCond : int  5 8 5 5 5 5 5 6 5 6 ...
 $ ExterQual   : chr  "Gd" "TA" "Gd" "TA" ...
 $ GrLivArea   : int  1710 1262 1786 1717 2198 1362 1694 2090 1774 1077 ...
 $ FullBath    : int  2 2 2 1 2 1 2 2 2 1 ...
 $ GarageArea  : int  548 460 608 642 836 480 636 484 468 205 ...
 $ BsmtCond    : chr  "TA" "TA" "TA" "Gd" ...
 $ FireplaceQu : chr  "NA" "TA" "TA" "Gd" ...
 $ Electrical  : chr  "SBrkr" "SBrkr" "SBrkr" "SBrkr" ...
 $ LotFrontage : num  65 80 68 60 84 85 75 0 51 50 ...
 $ KitchenQual : chr  "Gd" "TA" "Gd" "Gd" ...
 $ PavedDrive  : chr  "Y" "Y" "Y" "Y" ...
 $ SalePrice   : int  208500 181500 223500 140000 250000 14300

In [13]:
%%R
casas <- read.csv('Casas.csv')

str(casas)

'data.frame':	1460 obs. of  81 variables:
 $ Id           : int  1 2 3 4 5 6 7 8 9 10 ...
 $ MSSubClass   : int  60 20 60 70 60 50 20 60 50 190 ...
 $ MSZoning     : chr  "RL" "RL" "RL" "RL" ...
 $ LotFrontage  : int  65 80 68 60 84 85 75 NA 51 50 ...
 $ LotArea      : int  8450 9600 11250 9550 14260 14115 10084 10382 6120 7420 ...
 $ Street       : chr  "Pave" "Pave" "Pave" "Pave" ...
 $ Alley        : chr  NA NA NA NA ...
 $ LotShape     : chr  "Reg" "Reg" "IR1" "IR1" ...
 $ LandContour  : chr  "Lvl" "Lvl" "Lvl" "Lvl" ...
 $ Utilities    : chr  "AllPub" "AllPub" "AllPub" "AllPub" ...
 $ LotConfig    : chr  "Inside" "FR2" "Inside" "Corner" ...
 $ LandSlope    : chr  "Gtl" "Gtl" "Gtl" "Gtl" ...
 $ Neighborhood : chr  "CollgCr" "Veenker" "CollgCr" "Crawfor" ...
 $ Condition1   : chr  "Norm" "Feedr" "Norm" "Norm" ...
 $ Condition2   : chr  "Norm" "Norm" "Norm" "Norm" ...
 $ BldgType     : chr  "1Fam" "1Fam" "1Fam" "1Fam" ...
 $ HouseStyle   : chr  "2Story" "1Story" "2Story" "2Story" ...
